In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true;
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go

from dataclasses import dataclass
from enum import Enum
from IPython.display import display, Markdown
from plotly.subplots import make_subplots
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, auc
from typing import List, Tuple

In [ ]:
# user constants
CSVS_ROOT = "..."        # directory with csvs generated by ./generate_csvs_for_report.py
MODEL_NAME = "..."       # name of the evaluated model (only for naming in graphs)
MIN_ISLET_SIZE = 0       # [0, 50, 100, 150, 200]

In [ ]:
"""
is = incorrectly separated
fn = false negative
fp = false positive
"""
IMAGE_CSV_FILE_NAME = "min{}-image-stats.csv".format(MIN_ISLET_SIZE)
MATCHED_ISLETS_CSV_FILE_NAME = "min{}-matched-islets-stats.csv".format(MIN_ISLET_SIZE)
IS_ISLETS_CSV_FILE_NAME = "min{}-incorrectly-separated-islets-stats.csv".format(MIN_ISLET_SIZE)
FN_ISLETS_CSV_FILE_NAME = "min{}-false-negative-islets-stats.csv".format(MIN_ISLET_SIZE)
FP_ISLETS_CSV_FILE_NAME = "min{}-false-positive-islets-stats.csv".format(MIN_ISLET_SIZE)

MIN_ISLET_SIZES = [0, 50, 100, 150, 200, 250]
ISLET_SIZES_LABELS = ["0 - 50 μm", "50 - 100 μm", "100 - 150 μm", "150 - 200 μm", "200 - 250 μm", "> 250 μm"]
GT_NN_LABELS = ["GT", "NN"]

FN_COLOR = "#2F8045"
IS_GT_COLOR = "#3E986A"
MATCHED_GT_COLOR = "#7EB489"

FP_COLOR = "#F39F1D"
IS_NN_COLOR = "#F6B619"
MATCHED_NN_COLOR = "#FACD45"

OTHER_ISLETS_COLOR = "#D0D0D0"

transparent_bg = {
    "plot_bgcolor": "rgba(0, 0, 0, 0)",
}

In [ ]:
display(
    Markdown(
        "# Results for model: \"{}\" (min. islet size: {})".format(MODEL_NAME, MIN_ISLET_SIZE)
            )
)

In [ ]:
@dataclass
class DistributionGraphAttributes:
    yaxis: str
    offsetgroup: int
    legendgrouptitle_text: str
    
DISTRIBUTION_BAR_ATTRIBUTES = [
    DistributionGraphAttributes(
        yaxis="y1",
        offsetgroup="0",
        legendgrouptitle_text="GT islets",
    ),
    DistributionGraphAttributes(
        yaxis="y2",
        offsetgroup="1",
        legendgrouptitle_text="NN islets",
    ),
]

In [ ]:
def load_csvs() -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    image_df = pd.read_csv(os.path.join(CSVS_ROOT, IMAGE_CSV_FILE_NAME))
    matched_islets_df = pd.read_csv(os.path.join(CSVS_ROOT, MATCHED_ISLETS_CSV_FILE_NAME))
    is_islets_df = pd.read_csv(os.path.join(CSVS_ROOT, IS_ISLETS_CSV_FILE_NAME))
    fn_islets_df = pd.read_csv(os.path.join(CSVS_ROOT, FN_ISLETS_CSV_FILE_NAME))
    fp_islets_df = pd.read_csv(os.path.join(CSVS_ROOT, FP_ISLETS_CSV_FILE_NAME))
    
    return image_df, matched_islets_df, is_islets_df, fn_islets_df, fp_islets_df

In [ ]:
def get_relative_array(nn_values: np.array, gt_values: np.array) -> np.array:
    nn_values = nn_values[gt_values > 0]
    gt_values = gt_values[gt_values > 0]
    return nn_values/gt_values

In [ ]:
def get_updated_islet_size_list(size_list: list, size_um: float) -> List[int]:
    for i in range(len(MIN_ISLET_SIZES)):
        if i == len(MIN_ISLET_SIZES) - 1:
            size_list[i] += 1
            break
            
        if size_um < MIN_ISLET_SIZES[i+1]:
            size_list[i] += 1
            break
    return size_list

def get_islet_count_per_size_single(df: pd.DataFrame) -> List[int]:
    count_per_size = [0] * len(ISLET_SIZES_LABELS)
    
    for size in df.size_um.values:
        count_per_size = get_updated_islet_size_list(count_per_size, size)
        
    return count_per_size
        
def get_islet_count_per_size_both(df: pd.DataFrame) -> Tuple[List[int], List[int]]:
    count_per_size_nn = [0] * len(ISLET_SIZES_LABELS)
    count_per_size_gt = [0] * len(ISLET_SIZES_LABELS)
    
    for gt_size in df.drop_duplicates(subset=["image_name", "islet_id_gt"]).size_um_gt.values:
        count_per_size_gt = get_updated_islet_size_list(count_per_size_gt, gt_size)
        
    for nn_size in df.drop_duplicates(subset=["image_name", "islet_id_nn"]).size_um_nn.values:
        count_per_size_nn = get_updated_islet_size_list(count_per_size_nn, nn_size)
        
    return count_per_size_gt, count_per_size_nn

In [ ]:
def show_box_plot(boxes_values: List[List[float]], boxes_names: List[str], title: str, legend_position: str = "top"):
    layout = go.Layout(
        height=500,
        legend={
            "xanchor": "right",
            "x": 1,
            "yanchor": "bottom",
            "y": 1.01,
            "orientation": "h",
        },
        title={
            "text": title
        },
    )
    
    fig = go.Figure(layout=layout)
    for values, name in zip(boxes_values, boxes_names):
        fig.add_trace(go.Box(x=values, name=name))
        
    fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', minor_griddash="dot")

    fig.update_layout(transparent_bg)
    
    fig.show()

In [ ]:
def get_size_distribution_bar(name: str, y: List[int], color: str, legendgroup: int, show_gt_and_nn: bool = True) -> go.Bar:
    bar_attributes = DISTRIBUTION_BAR_ATTRIBUTES[legendgroup]
    return go.Bar(
        name=name, 
        x=ISLET_SIZES_LABELS, 
        y=y, 
        yaxis=bar_attributes.yaxis, 
        offsetgroup=bar_attributes.offsetgroup if show_gt_and_nn else 0, 
        offset=(legendgroup-1) * 1/3 if show_gt_and_nn else -0.4, 
        width=1/3 if show_gt_and_nn else 0.8, 
        marker_color=color,
        legendgroup=legendgroup, 
        legendgrouptitle_text=bar_attributes.legendgrouptitle_text
    )

def show_size_distriburion_chart(
    title: str,
    fn_count_per_size: List[int] = list(),
    fp_count_per_size: List[int] = list(),
    matched_gt_count_per_size: List[int] = list(),
    matched_nn_count_per_size: List[int] = list(),
    is_gt_count_per_size: List[int] = list(),
    is_nn_count_per_size: List[int] = list(),
    show_gt_and_nn: bool = True,
):
    fig = go.Figure(
        layout=go.Layout(
            barmode="relative",
            yaxis2=go.layout.YAxis(
                visible=False,
                matches="y",
                overlaying="y",
                anchor="x",
            ),
            height=600,
            title={"text": title}
        )

    )
    if len(fn_count_per_size) > 0:
        fig.add_trace(
            get_size_distribution_bar(name='False negative islets', y=fn_count_per_size, color=FN_COLOR, legendgroup=0, show_gt_and_nn=show_gt_and_nn)
        )
    if len(is_gt_count_per_size) > 0:
        fig.add_trace(
            get_size_distribution_bar(name='Incorrectly separated islets', y=is_gt_count_per_size, color=IS_GT_COLOR, legendgroup=0, show_gt_and_nn=show_gt_and_nn)
        )
    if len(matched_gt_count_per_size) > 0:
        fig.add_trace(
            get_size_distribution_bar(name='Matched islets', y=matched_gt_count_per_size, color=MATCHED_GT_COLOR, legendgroup=0, show_gt_and_nn=show_gt_and_nn)
        )
    
    if len(fp_count_per_size) > 0:
        fig.add_trace(
            get_size_distribution_bar(name='False positive islets', y=fp_count_per_size, color=FP_COLOR, legendgroup=1, show_gt_and_nn=show_gt_and_nn)
        )
    if len(is_nn_count_per_size) > 0:
        fig.add_trace(
            get_size_distribution_bar(name='Incorrectly separated islets', y=is_nn_count_per_size, color=IS_NN_COLOR, legendgroup=1, show_gt_and_nn=show_gt_and_nn)
        )
    if len(matched_nn_count_per_size) > 0:
        fig.add_trace(
            get_size_distribution_bar(name='Matched islets', y=matched_nn_count_per_size, color=MATCHED_NN_COLOR, legendgroup=1, show_gt_and_nn=show_gt_and_nn)
        )
    
    fig.show()

In [ ]:
def get_gt_nn_value_comparison_bar(
    name: str, value: int, color: str, legendgroup: int, showlegend: bool = True
) -> go.Bar:
    bar_attributes = DISTRIBUTION_BAR_ATTRIBUTES[legendgroup]
    return go.Bar(
        name=name,
        y=GT_NN_LABELS,
        x=[value if i == legendgroup else 0 for i in range(2)],
        marker_color=color,
        showlegend=showlegend,
        legendgroup=legendgroup, 
        legendgrouptitle_text=bar_attributes.legendgrouptitle_text, 
        orientation="h"
    )

def show_gt_nn_value_comparison_bar_chart(
    title: str,
    fn_islets_value: int = None,
    fp_islets_value: int = None,
    matched_gt_islets_value: int = None,
    matched_nn_islets_value: int = None,
    is_gt_islets_value: int = None,
    is_nn_islets_value: int = None,
    other_gt_islets_value: int = None,
    other_nn_islets_value: int = None,
    small: bool = False,
):
    fig = go.Figure(
        layout=go.Layout(
            barmode="stack",
            height=400 if not small else 320,
            title={"text": title},
        )
    )
    
    if fn_islets_value is not None:
        fig.add_trace(
            get_gt_nn_value_comparison_bar(name="False negative islets", value=fn_islets_value, color=FN_COLOR, legendgroup=0)
        )
    if is_gt_islets_value is not None:
        fig.add_trace(
            get_gt_nn_value_comparison_bar(name="Incorrectly separated islets", value=is_gt_islets_value, color=IS_GT_COLOR, legendgroup=0)
        )
    if matched_gt_islets_value is not None:
        fig.add_trace(
            get_gt_nn_value_comparison_bar(name="Matched islets", value=matched_gt_islets_value, color=MATCHED_GT_COLOR, legendgroup=0)
        )
    if other_gt_islets_value is not None:
        fig.add_trace(
            get_gt_nn_value_comparison_bar(name="Other GT islets", value=other_gt_islets_value, color=OTHER_ISLETS_COLOR, legendgroup=0)
        )
    
    if fp_islets_value is not None:
        fig.add_trace(
            get_gt_nn_value_comparison_bar(name="False positive islets", value=fp_islets_value, color=FP_COLOR, legendgroup=1)
        )
    if is_nn_islets_value is not None:
        fig.add_trace(
            get_gt_nn_value_comparison_bar(name="Incorrectly separated islets", value=is_nn_islets_value, color=IS_NN_COLOR, legendgroup=1)
        )
    if matched_nn_islets_value is not None:
        fig.add_trace(
            get_gt_nn_value_comparison_bar(name="Matched islets", value=matched_nn_islets_value, color=MATCHED_NN_COLOR, legendgroup=1)
        )
    if other_nn_islets_value is not None:
        fig.add_trace(
            get_gt_nn_value_comparison_bar(name="Other NN islets", value=other_nn_islets_value, color=OTHER_ISLETS_COLOR, legendgroup=1)
        )


    fig.show()
    
def add_gt_nn_value_comparison_bar(
    fig: go.Figure,
    row: int,
    fn_islets_value: int = None,
    fp_islets_value: int = None,
    matched_gt_islets_value: int = None,
    matched_nn_islets_value: int = None,
    is_gt_islets_value: int = None,
    is_nn_islets_value: int = None,
    other_gt_islets_value: int = None,
    other_nn_islets_value: int = None,
): 
    showlegend = row == 1
    
    if fn_islets_value is not None:
        fig.add_trace(
            get_gt_nn_value_comparison_bar(name="False negative islets", value=fn_islets_value, color=FN_COLOR, legendgroup=0, showlegend=showlegend),
            col=1, row=row
        )
    if is_gt_islets_value is not None:
        fig.add_trace(
            get_gt_nn_value_comparison_bar(name="Incorrectly separated islets", value=is_gt_islets_value, color=IS_GT_COLOR, legendgroup=0, showlegend=showlegend),
            col=1, row=row
        )
    if matched_gt_islets_value is not None:
        fig.add_trace(
            get_gt_nn_value_comparison_bar(name="Matched islets", value=matched_gt_islets_value, color=MATCHED_GT_COLOR, legendgroup=0, showlegend=showlegend),
            col=1, row=row
        )
    if other_gt_islets_value is not None:
        fig.add_trace(
            get_gt_nn_value_comparison_bar(name="Other GT islets", value=other_gt_islets_value, color=OTHER_ISLETS_COLOR, legendgroup=0, showlegend=showlegend),
            col=1, row=row
        )
    
    if fp_islets_value is not None:
        fig.add_trace(
            get_gt_nn_value_comparison_bar(name="False positive islets", value=fp_islets_value, color=FP_COLOR, legendgroup=1, showlegend=showlegend),
            col=1, row=row
        )
    if is_nn_islets_value is not None:
        fig.add_trace(
            get_gt_nn_value_comparison_bar(name="Incorrectly separated islets", value=is_nn_islets_value, color=IS_NN_COLOR, legendgroup=1, showlegend=showlegend),
            col=1, row=row
        )
    if matched_nn_islets_value is not None:
        fig.add_trace(
            get_gt_nn_value_comparison_bar(name="Matched islets", value=matched_nn_islets_value, color=MATCHED_NN_COLOR, legendgroup=1, showlegend=showlegend),
            col=1, row=row
        )
    if other_nn_islets_value is not None:
        fig.add_trace(
            get_gt_nn_value_comparison_bar(name="Other NN islets", value=other_nn_islets_value, color=OTHER_ISLETS_COLOR, legendgroup=1, showlegend=showlegend),
            col=1, row=row
        )
        
    return fig

def init_gt_nn_value_comparison_bar_chart():
    fig = make_subplots(
        rows=4, cols=1,
        row_heights=[0.25] * 4,
        specs=[[{"type": "bar"}], [{"type": "bar"}], [{"type": "bar"}], [{"type": "bar"}]],
        subplot_titles=[
            "Total islet count", 
            "Total islet area [μm]",
            "Total islet volume - Ellipse [IE]",
            "Total islet volume - Ricordi, short [IE]"  
        ],
        vertical_spacing=0.1,
        figure=go.Figure(
            layout = go.Layout(
                height=1000,
                barmode="stack",
            )
        )
    )
    return fig

In [ ]:
class CorrGraphValueType(Enum):
    COUNT = {"name": "count", "unit": "Number of islets", "unit_short": "islets"},
    AREA = {"name": "area", "unit": "μm\u00b2", "unit_short": "μm\u00b2"},
    VOLUME_ELLIPSE = {"name": "volume (Ellipse)", "unit": "IE", "unit_short": "IE"},
    VOLUME_RICORDI_SHORT = {"name": "volume (Ricordi, short)", "unit": "IE", "unit_short": "IE"},
    
@dataclass
class RegressionResult:
    coef: float
    intercept: float
    r2: float
    

def fit_regression_line(x: np.array, y: np.array) -> RegressionResult:
    model = LinearRegression().fit(x.reshape((-1, 1)), y)
    r2 = r2_score(x, y)
    return RegressionResult(model.coef_[0], model.intercept_, r2)

def add_relative_dist_histogram(fig: go.Figure, gt: np.array, nn: np.array, scope: str):
    rel_array = get_relative_array(nn, gt)
    mean_value = np.mean(rel_array)
    
    fig.add_trace(
        go.Histogram(x=rel_array, nbinsx=50, marker_color="#636EFA", legendgroup=0, showlegend=False), 
        row=1, col=1
    )
    fig.add_vline(x=mean_value, line_dash="dash", line=dict(color='red',), row=1, col=1)
    fig = add_histogram_annotations(fig, rel_array, scope)
    
    return fig

def add_correlation_graph(fig: go.Figure, gt: np.array, nn: np.array, scope: str, value_type: CorrGraphValueType):
    regression_result = fit_regression_line(gt, nn)
    max_value = max(max(gt), max(nn))
    
    fig.add_trace(go.Scatter(x=gt, y=nn, mode='markers', marker_color="#636EFA", legendgroup=0, showlegend=False), row=1, col=2)
    
    fig.add_trace(
        go.Scatter(
            x=[0, max_value], 
            y=[regression_result.intercept, max_value * regression_result.coef + regression_result.intercept], 
            mode="lines", 
            name="fitted", 
            line=dict(color='red', dash='solid'), 
            legendgroup=1,
        ),
        row=1, col=2,
    )
    fig.add_trace(
        go.Scatter(
            x=[0, max_value], 
            y=[0, max_value], 
            mode="lines", 
            name="optimal", 
            line=dict(color='grey', dash='dash'), 
            legendgroup=1,
        ),
        row=1, col=2,
    )
    fig.update_layout(
        xaxis2=dict(range=[0, max_value * 1.1]),
        yaxis2=dict(range=[0, max_value * 1.1]),
    )
    fig = add_correlation_graph_annotations(fig, gt, nn, regression_result, scope, value_type)
        
    return fig

def get_histogram_annotation(text: str, align: str, x: float, y: float) -> go.layout.Annotation:
    return go.layout.Annotation(
                text=text,
                font={"size": 10},
                align=align,
                showarrow=False,
                xref='x domain',
                yref='y domain',
                x=x,
                y=y,
    )


def add_histogram_annotations(fig: go.Figure, rel_array: np.array, scope: str) -> go.Figure:
    header = "{} {}s".format(len(rel_array), scope)
    right_values = "Min. islet size:<br>Median:<br>Mean:<br>Std:"
    left_values = "0 μm<br>{:.2f}<br>{:.2f}<br>{:.2f}".format(np.median(rel_array), np.mean(rel_array), np.std(rel_array))
    
    annotations = [
        get_histogram_annotation(text=header, align="right", x=0.98, y=0.98),
        get_histogram_annotation(text=right_values, align="left", x=0.90, y=0.94),
        get_histogram_annotation(text=left_values, align="right", x=0.98, y=0.94),
    ]
    
    for annotation in annotations:
        fig.add_annotation(annotation)
        
    return fig

def get_correlation_graph_annotation(text: str, align: str, x: float, y: float) -> go.layout.Annotation:
    return go.layout.Annotation(
                text=text,
                font={"size": 10},
                align=align,
                showarrow=False,
                xref='x2 domain',
                yref='y2 domain',
                x=x,
                y=y,
    )

def add_correlation_graph_annotations(
    fig: go.Figure, 
    gt: np.array, 
    nn: np.array, 
    regression_result: RegressionResult, 
    scope: str,
    value_type: CorrGraphValueType
) -> go.Figure:
    sign = '+' if regression_result.intercept >= 0 else '-'
    text_regression = "y = {:.2f}x {} {:.2f}<br>R<sup>2</sup> = {:.2f}".format(
        regression_result.coef, sign, abs(regression_result.intercept), regression_result.r2
    )
    header =  "Total islet {} on {} {}s".format(value_type.value[0]["name"], len(gt), scope)
    right_values = "GT:<br>NN:<br>Min. islet size:"
    if value_type == CorrGraphValueType.COUNT:
        left_values = "{0} {2}<br>{1} {2}<br>0 μm".format(np.sum(gt), np.sum(nn), value_type.value[0]["unit_short"])
    else:
        left_values = "{0:.2f} {2}<br>{1:.2f} {2}<br>0 μm".format(np.sum(gt), np.sum(nn), value_type.value[0]["unit_short"])
    
    annotations = [
        get_correlation_graph_annotation(text=text_regression, align="left", x=0.02, y=0.98),
        get_correlation_graph_annotation(text=header, align="right", x=0.98, y=0.14),
        get_correlation_graph_annotation(text=right_values, align="left", x=0.80, y=0.02),
        get_correlation_graph_annotation(text=left_values, align="right", x=0.98, y=0.02),
    ]
    
    for annotation in annotations:
        fig.add_annotation(annotation)
        
    return fig

def show_correlation_graph_with_histogram(gt: np.array, nn: np.array, value_type: CorrGraphValueType, scope: str):
    fig = make_subplots(
        rows=1, cols=2,
        column_widths=[0.55, 0.45],
        specs=[[{"type": "histogram"}, {"type": "scatter"}]],
        subplot_titles=[
            "Distribution of relative islet {}".format(value_type.value[0]["name"]), 
            "GT/NN islet {} correlation".format(value_type.value[0]["name"])
        ],
        horizontal_spacing=0.1,
        figure=go.Figure(
            layout = go.Layout(
                width=1050,
                height=500,
                title="Islet {} per image".format(value_type.value[0]["name"]),
                legend={
                    "xanchor": "right",
                    "x": 1,
                    "yanchor": "top",
                    "y": 1,
                    "orientation": "v",
                },
            )
        )
    )
    fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, ticks="outside")
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, ticks="outside")

    fig.update_layout(
        xaxis_title="Relative error (NN/GT)",
        yaxis_title="{} count".format(scope.capitalize()),
        xaxis2_title="GT [{}]".format(value_type.value[0]["unit"]),
        yaxis2_title="NN [{}]".format(value_type.value[0]["unit"]),
    )

    fig.update_layout(transparent_bg)
    
    fig = add_relative_dist_histogram(fig, gt, nn, scope)
    fig = add_correlation_graph(fig, gt, nn, scope, value_type)
    
    fig.show()

In [ ]:
def get_max_iou_per_gt_islet(df: pd.DataFrame):
    max_iou_per_gt_islet = list()

    for image_name in df.image_name.unique():
        sub_df = df.loc[df.image_name == image_name]

        for gt_id in sub_df.islet_id_gt.unique():
            max_iou = sub_df.loc[sub_df.islet_id_gt == gt_id].iou.max()
            max_iou_per_gt_islet.append(max_iou)

    max_iou_per_gt_islet.sort(reverse=True)
    return max_iou_per_gt_islet

def plot_gt_ratio_iou_curve(max_iou_per_gt_islet: List[float], gt_islets_ratio: List[float]):
    fig = go.Figure(layout=go.Layout(
                legend={
                    "xanchor": "right",
                    "x": 1,
                    "yanchor": "top",
                    "y": 1,
                    "orientation": "v",
                }
    ))
    
    fig.add_trace(go.Scatter(
        x=max_iou_per_gt_islet, 
        y=gt_islets_ratio, 
        mode='lines',
    ))
    
    fig.update_layout(
        title=f"Ratio of GT islets with max IoU higher than threshold (AUC={auc(max_iou_per_gt_islet, gt_islets_ratio):.4f})",#"Ratio of incorrectly separated GT islets with max IoU higher than threshold ",
        xaxis_title="IoU threshold",
        yaxis_title="Ratio of GT islets",#"Ratio of incorrectly separated GT islets",
    )
    fig.show()
    
def plot_max_iou_per_gt_distribution(max_iou_per_gt_islet: List[float]):
    fig = make_subplots(
        rows=2, cols=1,
        row_heights=[0.2, 0.8],
        specs=[[{"type": "box"}], [{"type": "histogram"}]],
        vertical_spacing=0.05,
        figure=go.Figure(
            layout = go.Layout(
                height=700,
                showlegend=False,
                title="Distribution of max IoU per GT islet"
            )
        )
    )


    fig.add_trace(go.Histogram(x=max_iou_per_gt_islet, nbinsx=50, marker_color="#636EFA"), row=2, col=1)
    fig.add_trace(go.Box(x=max_iou_per_gt_islet, name='', marker_color="#636EFA", orientation='h'), row=1, col=1)
    fig.update_layout(
        xaxis2_title="Max IoU",
        yaxis2_title="Number of GT islets"
    )
    
    fig.show()

def show_gt_max_iou_graphs(df: pd.DataFrame):
    max_iou_per_gt_islet =  get_max_iou_per_gt_islet(df)
    num_gt_islets = len(max_iou_per_gt_islet)

    y_data = [float(sum((np.array(max_iou_per_gt_islet) >= 1)))] + [(i)/num_gt_islets for i in range(num_gt_islets)] + [1.0]
    max_iou_per_gt_islet_extended = [1.0] + max_iou_per_gt_islet + [0.0]
    
    plot_gt_ratio_iou_curve(max_iou_per_gt_islet_extended, y_data)
    plot_max_iou_per_gt_distribution(max_iou_per_gt_islet)
    

In [ ]:
image_df, matched_islets_df, is_islets_df, fn_islets_df, fp_islets_df = load_csvs()

In [ ]:
fn_islet_count_per_size = get_islet_count_per_size_single(fn_islets_df)
fp_islet_count_per_size = get_islet_count_per_size_single(fp_islets_df)
matched_gt_islet_count_per_size, matched_nn_islet_count_per_size = get_islet_count_per_size_both(matched_islets_df)
is_gt_islet_count_per_size, is_nn_islet_count_per_size = get_islet_count_per_size_both(is_islets_df)

fn_islets_count = np.sum(fn_islet_count_per_size)
fp_islets_count = np.sum(fp_islet_count_per_size)
matched_gt_islets_count = np.sum(matched_gt_islet_count_per_size)
matched_nn_islets_count = np.sum(matched_gt_islet_count_per_size)
is_gt_islets_count = np.sum(is_gt_islet_count_per_size)
is_nn_islets_count = np.sum(is_nn_islet_count_per_size)

fn_islets_total_area = sum(fn_islets_df.area_um2.values)
fp_islets_total_area = sum(fp_islets_df.area_um2.values)
is_gt_islets_total_area = sum(is_islets_df.drop_duplicates(subset=["image_name", "islet_id_gt"]).area_um2_gt.values)
is_nn_islets_total_area = sum(is_islets_df.drop_duplicates(subset=["image_name", "islet_id_nn"]).area_um2_nn.values)
matched_gt_islets_total_area = sum(matched_islets_df.area_um2_gt.values)
matched_nn_islets_total_area = sum(matched_islets_df.area_um2_nn.values)

fn_islets_total_volume_ellipse = sum(fn_islets_df.volume_ellipse_ie.values)
fp_islets_total_volume_ellipse = sum(fp_islets_df.volume_ellipse_ie.values)
is_gt_islets_total_volume_ellipse = sum(is_islets_df.drop_duplicates(subset=["image_name", "islet_id_gt"]).volume_ellipse_ie_gt.values)
is_nn_islets_total_volume_ellipse = sum(is_islets_df.drop_duplicates(subset=["image_name", "islet_id_nn"]).volume_ellipse_ie_nn.values)
matched_gt_islets_total_volume_ellipse = sum(matched_islets_df.volume_ellipse_ie_gt.values)
matched_nn_islets_total_volume_ellipse = sum(matched_islets_df.volume_ellipse_ie_nn.values)

fn_islets_total_volume_ricordi_short = sum(fn_islets_df.volume_ricordi_short_ie.values)
fp_islets_total_volume_ricordi_short = sum(fp_islets_df.volume_ricordi_short_ie.values)
is_gt_islets_total_volume_ricordi_short = sum(is_islets_df.drop_duplicates(subset=["image_name", "islet_id_gt"]).volume_ricordi_short_ie_gt.values)
is_nn_islets_total_volume_ricordi_short = sum(is_islets_df.drop_duplicates(subset=["image_name", "islet_id_nn"]).volume_ricordi_short_ie_nn.values)
matched_gt_islets_total_volume_ricordi_short = sum(matched_islets_df.volume_ricordi_short_ie_gt.values)
matched_nn_islets_total_volume_ricordi_short = sum(matched_islets_df.volume_ricordi_short_ie_nn.values)

## 1. Overall results

### 1.1 Relative metrics

In [ ]:
attributes = ["islet_count", "total_area_um2", "total_volume_ellipse_ie", "total_volume_ricordi_short_ie"]
boxes_names = [
    "Relative islet count", 
    "Relative islet area", 
    "Relative islet volume<br> (Ellipse)", 
    "Relative islet volume<br> (Ricordi, short)"
]

boxes_values = list()
for attribute in attributes:
    relative_values = get_relative_array(
        nn_values=image_df["{}_nn".format(attribute)],
        gt_values=image_df["{}_gt".format(attribute)],
    )
    boxes_values.append(relative_values)

show_box_plot(boxes_values, boxes_names, "Relative values (NN/GT)")

### 1.2 Semantic metrics

In [ ]:
boxes_names = ["Dice score", "IoU", "Precision", "Recall"]

boxes_values = [
    image_df.dice_score,
    image_df.iou,
    image_df.precision,
    image_df.recall,
]

print("IoU: {} ± {}".format(image_df.iou.mean(), image_df.iou.std()))
show_box_plot(boxes_values, boxes_names, "Metrics", legend_position="bottom")

### 1.3 Correlation graphs

In [ ]:
show_correlation_graph_with_histogram(
    gt=image_df.islet_count_gt.values, 
    nn=image_df.islet_count_nn.values,
    value_type=CorrGraphValueType.COUNT,
    scope="image",
)

show_correlation_graph_with_histogram(
    gt=image_df.total_area_um2_gt.values, 
    nn=image_df.total_area_um2_nn.values,
    value_type=CorrGraphValueType.AREA,
    scope="image",
)

show_correlation_graph_with_histogram(
    gt=image_df.total_volume_ellipse_ie_gt.values, 
    nn=image_df.total_volume_ellipse_ie_nn.values,
    value_type=CorrGraphValueType.VOLUME_ELLIPSE,
    scope="image",
)

show_correlation_graph_with_histogram(
    gt=image_df.total_volume_ricordi_short_ie_gt.values, 
    nn=image_df.total_volume_ricordi_short_ie_nn.values,
    value_type=CorrGraphValueType.VOLUME_RICORDI_SHORT,
    scope="image",
)


### 1.4 Size distribution

In [ ]:
show_size_distriburion_chart(
    "Islet sizes distribution",
    fn_islet_count_per_size,
    fp_islet_count_per_size,
    matched_gt_islet_count_per_size,
    matched_nn_islet_count_per_size,
    is_gt_islet_count_per_size,
    is_nn_islet_count_per_size,
)

### 1.5 Islet counts

In [ ]:
show_gt_nn_value_comparison_bar_chart(
    "Total islet count",
    fn_islets_value=fn_islets_count,
    fp_islets_value=fp_islets_count,
    matched_gt_islets_value=matched_gt_islets_count,
    matched_nn_islets_value=matched_nn_islets_count,
    is_gt_islets_value=is_gt_islets_count,
    is_nn_islets_value=is_nn_islets_count,
)

## 2. Matched islets

### 2.1 Correlation graphs

In [ ]:
show_correlation_graph_with_histogram(
    gt=matched_islets_df.area_um2_gt.values, 
    nn=matched_islets_df.area_um2_nn.values,
    value_type=CorrGraphValueType.AREA,
    scope="islet",
)

show_correlation_graph_with_histogram(
    gt=matched_islets_df.volume_ellipse_ie_gt.values, 
    nn=matched_islets_df.volume_ellipse_ie_nn.values,
    value_type=CorrGraphValueType.VOLUME_ELLIPSE,
    scope="islet",
)

show_correlation_graph_with_histogram(
    gt=matched_islets_df.volume_ricordi_short_ie_gt.values, 
    nn=matched_islets_df.volume_ricordi_short_ie_nn.values,
    value_type=CorrGraphValueType.VOLUME_RICORDI_SHORT,
    scope="islet",
)

### 2.2 Size distribution

In [ ]:
show_size_distriburion_chart(
    "Matched islet sizes distribution",
    matched_gt_count_per_size=matched_gt_islet_count_per_size,
    matched_nn_count_per_size=matched_nn_islet_count_per_size,
)

### 2.3 Total count, area, volume

In [ ]:
fig = init_gt_nn_value_comparison_bar_chart()

fig = add_gt_nn_value_comparison_bar(
    fig,
    row=1,
    matched_gt_islets_value=matched_gt_islets_count,
    matched_nn_islets_value=matched_nn_islets_count,
    other_gt_islets_value=fn_islets_count + is_gt_islets_count,
    other_nn_islets_value=fp_islets_count + is_nn_islets_count,
)

fig = add_gt_nn_value_comparison_bar(
    fig,
    row=2,
    matched_gt_islets_value=matched_gt_islets_total_area,
    matched_nn_islets_value=matched_nn_islets_total_area,
    other_gt_islets_value=fn_islets_total_area + is_gt_islets_total_area,
    other_nn_islets_value=fp_islets_total_area + is_nn_islets_total_area,
)

fig = add_gt_nn_value_comparison_bar(
    fig,
    row=3,
    matched_gt_islets_value=matched_gt_islets_total_volume_ellipse,
    matched_nn_islets_value=matched_nn_islets_total_volume_ellipse,
    other_gt_islets_value=fn_islets_total_volume_ellipse + is_gt_islets_total_volume_ellipse,
    other_nn_islets_value=fp_islets_total_volume_ellipse + is_nn_islets_total_volume_ellipse,
)

fig = add_gt_nn_value_comparison_bar(
    fig,
    row=4,
    matched_gt_islets_value=matched_gt_islets_total_volume_ricordi_short,
    matched_nn_islets_value=matched_nn_islets_total_volume_ricordi_short,
    other_gt_islets_value=fn_islets_total_volume_ricordi_short + is_gt_islets_total_volume_ricordi_short,
    other_nn_islets_value=fp_islets_total_volume_ricordi_short + is_nn_islets_total_volume_ricordi_short,
)

fig.show()

## 3. Incorrectly separated islets

### 3.1 Correlation graphs

In [ ]:
is_gt_grouped_by_image = is_islets_df.drop_duplicates(subset=["image_name", "islet_id_gt"]).groupby(by=["image_name"]).sum()
is_nn_grouped_by_image = is_islets_df.drop_duplicates(subset=["image_name", "islet_id_nn"]).groupby(by=["image_name"]).sum()

In [ ]:
show_correlation_graph_with_histogram(
    gt=image_df.incorrectly_separated_islets_count_gt.values, 
    nn=image_df.incorrectly_separated_islets_count_nn.values,
    value_type=CorrGraphValueType.COUNT,
    scope="image",
)

show_correlation_graph_with_histogram(
    gt=is_gt_grouped_by_image.area_um2_gt.values, 
    nn=is_nn_grouped_by_image.area_um2_nn.values,
    value_type=CorrGraphValueType.AREA,
    scope="image",
)


In [ ]:
show_correlation_graph_with_histogram(
    gt=is_gt_grouped_by_image.volume_ellipse_ie_gt.values, 
    nn=is_nn_grouped_by_image.volume_ellipse_ie_nn.values,
    value_type=CorrGraphValueType.VOLUME_ELLIPSE,
    scope="image",
)

show_correlation_graph_with_histogram(
    gt=is_gt_grouped_by_image.volume_ricordi_short_ie_gt.values, 
    nn=is_nn_grouped_by_image.volume_ricordi_short_ie_nn.values,
    value_type=CorrGraphValueType.VOLUME_RICORDI_SHORT,
    scope="image",
)

### 3.2 GT islets max IoU

In [ ]:
show_gt_max_iou_graphs(is_islets_df)

### 3.3 SIze distribution

In [ ]:
show_size_distriburion_chart(
    "Incorrectly separated islet sizes distribution",
    is_gt_count_per_size=is_gt_islet_count_per_size,
    is_nn_count_per_size=is_nn_islet_count_per_size,
)

### 3.4 Total count, area, volume

In [ ]:
fig = init_gt_nn_value_comparison_bar_chart()

fig = add_gt_nn_value_comparison_bar(
    fig,
    row=1,
    is_gt_islets_value=is_gt_islets_count,
    is_nn_islets_value=is_nn_islets_count,
    other_gt_islets_value=fn_islets_count + matched_gt_islets_count,
    other_nn_islets_value=fp_islets_count + matched_nn_islets_count,
)

fig = add_gt_nn_value_comparison_bar(
    fig,
    row=2,
    is_gt_islets_value=is_gt_islets_total_area,
    is_nn_islets_value=is_nn_islets_total_area,
    other_gt_islets_value=fn_islets_total_area + matched_gt_islets_total_area,
    other_nn_islets_value=fp_islets_total_area + matched_nn_islets_total_area,
)

fig = add_gt_nn_value_comparison_bar(
    fig,
    row=3,
    is_gt_islets_value=is_gt_islets_total_volume_ellipse,
    is_nn_islets_value=is_nn_islets_total_volume_ellipse,
    other_gt_islets_value=fn_islets_total_volume_ellipse + matched_gt_islets_total_volume_ellipse,
    other_nn_islets_value=fp_islets_total_volume_ellipse + matched_nn_islets_total_volume_ellipse,
)

fig = add_gt_nn_value_comparison_bar(
    fig,
    row=4,
    is_gt_islets_value=is_gt_islets_total_volume_ricordi_short,
    is_nn_islets_value=is_nn_islets_total_volume_ricordi_short,
    other_gt_islets_value=fn_islets_total_volume_ricordi_short + matched_gt_islets_total_volume_ricordi_short,
    other_nn_islets_value=fp_islets_total_volume_ricordi_short + matched_nn_islets_total_volume_ricordi_short,
)

fig.show()

## 4. False negative/positive islets

### 4.1 Size distribution

In [ ]:
show_size_distriburion_chart(
    "False negative/positive islet sizes distribution",
    fn_count_per_size=fn_islet_count_per_size,
    fp_count_per_size=fp_islet_count_per_size,
)

### 4.2 Total count, area, volume

In [ ]:
fig = init_gt_nn_value_comparison_bar_chart()

fig = add_gt_nn_value_comparison_bar(
    fig,
    row=1,
    fn_islets_value=fn_islets_count,
    fp_islets_value=fp_islets_count,
    other_gt_islets_value=matched_gt_islets_count + is_gt_islets_count,
    other_nn_islets_value=matched_gt_islets_count + is_gt_islets_count,
)

fig = add_gt_nn_value_comparison_bar(
    fig,
    row=2,
    fn_islets_value=fn_islets_total_area,
    fp_islets_value=fp_islets_total_area,
    other_gt_islets_value=matched_gt_islets_total_area + is_gt_islets_total_area,
    other_nn_islets_value=matched_nn_islets_total_area + is_gt_islets_total_area,
)

fig = add_gt_nn_value_comparison_bar(
    fig,
    row=3,
    fn_islets_value=fn_islets_total_volume_ellipse,
    fp_islets_value=fp_islets_total_volume_ellipse,
    other_gt_islets_value=matched_gt_islets_total_volume_ellipse + is_gt_islets_total_volume_ellipse,
    other_nn_islets_value=matched_nn_islets_total_volume_ellipse + is_gt_islets_total_volume_ellipse,
)

fig = add_gt_nn_value_comparison_bar(
    fig,
    row=4,
    fn_islets_value=fn_islets_total_volume_ricordi_short,
    fp_islets_value=fp_islets_total_volume_ricordi_short,
    other_gt_islets_value=matched_gt_islets_total_volume_ricordi_short + is_gt_islets_total_volume_ricordi_short,
    other_nn_islets_value=matched_nn_islets_total_volume_ricordi_short + is_gt_islets_total_volume_ricordi_short,
)

fig.show()